# Transfer Learning usando lasagne - parte 1

Nesse tutorial, vamos utilizar CNNs treinadas na base ImageNet para outros problemas de classificação de imagens, usando transfer learning.

O objetivo é treinar um modelo para discriminar um subconjunto de 5 classes da base de dados Caltech 101 (http://www.vision.caltech.edu/Image_Datasets/Caltech101/). A base para esse exercício possui 325 imagens de tamanho 224x224x3. 

Na parte 1 desse tutorial, vamos utilizar o método "DeCAF" descrito em [1]: 
 1. Vamos utilizar uma rede treinada na base ImageNet: https://github.com/Lasagne/Recipes/tree/master/modelzoo
 2. Usaremos essa rede para "extrair características" da base Caltech, aplicando forward-propagation e obtendo as ativações em uma das últimas camadas da rede
 3. Vamos treinar modelos lineares utilizando essa representação


[1] Jeff Donahue et al., “DeCAF: A Deep Convolutional Activation Feature for Generic Visual Recognition,” arXiv:1310.1531 [Cs], October 5, 2013, http://arxiv.org/abs/1310.1531.

In [ ]:
import numpy as np
import urllib  # Para baixar o modelo

import lasagne
import theano
import cPickle # Para carregar o modelo do disco
import matplotlib.pyplot as plt # Para visualizações
import os
%matplotlib inline

In [ ]:
# Começamos carregando a base de dados:

if not os.path.exists('caltech_5classes.npz'):
    print 'Baixando base de dados'
    urllib.urlretrieve('http://www.inf.ufpr.br/lghafemann/caltech_5classes.npz', 'caltech_5classes.npz')
    print 'Concluído'

data = np.load('caltech_5classes.npz')
x_train = data['x_train']
y_train = data['y_train']
x_test = data['x_test']
y_test = data['y_test']
classes = data['classes']

In [ ]:
x_train.shape

In [ ]:
#classes:
print classes

In [ ]:
#Mostrando alguns exemplos da base de dados

f, ax = plt.subplots(3,3, figsize=(8,8))
random_idx = np.random.choice(len(x_train), 9)
for i in range(3):
    for j in range(3):
        idx = random_idx[i*3+j]
        ax[i][j].imshow(x_train[idx])
        ax[i][j].axis('off')
        ax[i][j].set_title(classes[y_train[idx]])
f.suptitle('Exemplos da base de dados', fontsize=20)

A base de dados de treinamento portanto possui 5 classes('caranguejo' 'lagostim' 'crocodilo' 'dalmata' 'golfinho'), e um total de  260 imagens de tamanho 224x224, com 3 canais (imagem colorida - RGB).

Notamos que essa base de dados é muito mais complexa do que trabalhamos anteriormente. Nesse exercício, iremos utilizar uma rede convolucional treinada na base Imagenet para extraírmos características nessa base.

Vamos primeiro separar a base de dados em treinamento e validação. Podemos fazer isso utilizando a seguinte função do pacote 'scikit-learn':

```
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=fracao_de_teste)
```

Essa função divide uma base de dados em treinamento e teste (ou treinamento e validação, no nosso caso). O parâmetro "test_size" representa a fração de exemplos a ser usado para teste (deve ser entre 0 e 1). Vamos criar uma base de validação com 20% dos exemplos (e portanto, 80% para treinamento):

In [ ]:
from sklearn.cross_validation import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [ ]:
print 'Base de treino: ', x_train.shape
print 'Base de validacao: ', x_valid.shape

## Baseline: Regressão logística diretamente nos pixels

Para termos uma referência para comparação, vamos treinar um modelo de regressão logística utilizando diretamente os pixels como entrada.

Para tanto, vamos considerar cada exemplo como um vetor de tamanho 224x224x3 = 150528 dimensões:


In [ ]:
x_train_flat = x_train.reshape(len(x_train), -1)
x_valid_flat = x_valid.reshape(len(x_valid), -1)

print x_train_flat.shape

## Exercício: aplicação de regressão logística

Nesse exercício, vamos utilizar a biblioteca scikit-learn para treinar e verificar um modelo de regressão logística.
Essa biblioteca possui implementação de vários algoritmos de aprendizagem de máquina, com uma interface fácil de usar.

Para esse exercício, vamos treinar um modelo usando x_train_flat e y_train, e avaliar a performance na base de validação. Para isso, utilize a classe ```sklearn.linear_model.LogisticRegression``` ([manual](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression))

Dica: verifique a seção "Methods" do manual, em especial os métodos "fit" e "score")

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# Sua solução
classifier = LogisticRegression()

# Codigo para treinar o modelo e verificar a performance

In [ ]:
%load solutions/transfer_linear.py

Verificamos que a performance é ~ 50%. Para uma base de dados com 5 classes, onde um classificador aleatório está certo 20% das vezes, essa é uma taxa bem baixa.

# Utilizando um modelo pré-treinado na biblioteca Lasagne

Treinar um modelo grande na base ImageNet (que contém milhões de imagens) requer vários dias (ou semanas) mesmo utilizando boas GPUs em paralelo. Felizmente, alguns autores disponibilizam suas redes treinadas online, para facilitar a pesquisa de outras pessoas.

Para a biblioteca lasagne, tais modelos podem ser encontrados nesse link:

https://github.com/Lasagne/Recipes/tree/master/modelzoo

À seguir, vamos utilizar uma dessas redes, conhecida como vgg_cnn_s (publicada em [2]). Para tanto, precisamos baixar tanto a definição do modelo (um arquivo .py que implementa a arquitetura), quanto os pesos (em um arquivo "pickle").

[2] Karen Simonyan and Andrew Zisserman, “Very Deep Convolutional Networks for Large-Scale Image Recognition,” arXiv:1409.1556 [Cs], September 4, 2014, http://arxiv.org/abs/1409.1556.

## Obtendo a arquitetura do modelo

Execute as células abaixo para baixar o modelo vgg_cnn_s.py e carregar o seu conteúdo para esse notebook (se você encontrar um erro, leia as células seguintes)

In [ ]:
if not os.path.exists('vgg_cnn_s.py'):
    print 'Baixando a definição do modelo'
    urllib.urlretrieve('https://raw.githubusercontent.com/Lasagne/Recipes/master/modelzoo/vgg_cnn_s.py', 'vgg_cnn_s.py')
    print 'OK'

In [ ]:
%load vgg_cnn_s.py

O arquivo disponibilizado contém a definição do modelo. Notamos que o modelo possui 8 camadas, contendo camadas convolucionais, max-pooling e camadas fully-connected, que utilizamos ontem, assim como outras camadas (dropout, Local Response Normalization).

## Importante:

Ao tentar executar o código acima, vamos obter um erro, pois o modelo espera o uso de GPUs, que não estão disponíveis nesses computadores. Para utilizarmos esse modelo, precisamos fazer uma pequena alteração no código acima:

## Exercício: 

Altere na célula acima a linha que importa a camada Conv2DLayer, para utilizar a versão que funciona tanto em CPU quanto GPU. Isto é, altere:

```
from lasagne.layers.dnn import Conv2DDNNLayer as ConvLayer
```

para

```
from lasagne.layers import Conv2DLayer as ConvLayer
```

Em seguida, execute a célula acima. Vamos agora criar esse modelo:

In [ ]:
model = build_model()

Vamos analizar algumas propriedades desse modelo, como quais camadas ele possui, quantos parâmetros, etc.

In [ ]:
# Lista de camadas disponíveis:
model.keys()

Onde a camada de entrada é a "input", e a camada de saída chama-se "prob".

Para verificarmos o tamanho da entrada, podemos verificar a propriedade ```model['input'].shape```

Para verificarmos a saída de uma determinada camada, podemos usar a seguinte função:

```
tamanho = lasagne.layers.get_output_shape(camada)
```



In [ ]:
#Tamanho da entrada e da saída:

print 'Tamanho da entrada: ', model['input'].shape
print 'Tamanho da camada fc7: ', lasagne.layers.get_output_shape(model['fc7'])
print 'Tamanho da camada prob: ', lasagne.layers.get_output_shape(model['prob'])

Portanto, a entrada é um conjunto de imagens de tamanho 224x224, com 3 canais. A saída possui 1000 classes (as 1000 classes da base de dados ImageNet), e a penúltima camada possui dimensão 4096.

In [ ]:
# Número de parâmetros do modelo:

print lasagne.layers.count_params(model['prob'])

O modelo possui mais de 102 milhões de parâmetros(!).

## Baixando os parâmetros treinados na base Imagenet

Acima, nós carregamos a definição do modelo vgg_cnn_s, mas esse modelo foi criado com pesos aleatórios. Vamos agora fazer o download dos pesos (conforme definido no arquivo vgg_cnn_s.py):

In [ ]:
if not os.path.exists('vgg_cnn_s.pkl'):
    print 'Baixando pesos (394MB)'
    urllib.urlretrieve('https://s3.amazonaws.com/lasagne/recipes/pretrained/imagenet/vgg_cnn_s.pkl', 'vgg_cnn_s.pkl' )
    print 'OK'

In [ ]:
params = cPickle.load(open('vgg_cnn_s.pkl'))

In [ ]:
params.keys()

O arquivo baixado possui três informações:

* values - a lista de parâmetros treinados
* synset words - nome das 1000 classes da base Imagenet
* mean image - a imagem média que foi utilizada para treinamento

Esse terceiro parâmetro será explicado em mais detalhes abaixo. Por hora, vamos utilizar o primeiro parâmetro para carregar os pesos treinados no nosso modelo, usando a função

```
lasagne.layers.set_all_param_values(camada_de_saida, parametros)
```

In [ ]:
lasagne.layers.set_all_param_values(model['prob'], params['values'])

## Pré-processamento

**importante:** Em Transfer Learning, além de utilizar a rede com os mesmos pesos, é importante utilizarmos as mesmas etapas de pré-processamento que foram utilizados na base de dados origem antes do treinamento da rede neural. 

Em geral, um dos pré-processamentos amplamente utilizados para treinamento de redes neurais é o chamado "normalização da média" (mean normalization). Esse processo refere-se a modificar a base de treinamento para que cada dimensão possua média 0 ao longo da base de treinamento. Isso é feito da seguinte forma:

```
media = X.mean(axis=0)
X = X - media
```

Para novas imagens é importante seguir o mesmo procedimento (subtrair a média de cada dimensão). Para isso, vamos usar a media estimada na base de treinamento, que é dada por ```params['mean image']```. 

Além disso, precisamos tomar outras duas ações:

1. A nossa base de dados possui tamanho (208, 224, 224, 3): Número de exemplos x Altura x Largura x Canais RGB. A entrada da rede é (None, 3, 224, 224). Precisamos portanto modificar a ordem das dimensões. Podemos fazer isso usando a função ```np.transpose```
2. A base de dados possui canais RGB (vermelho, verde, azul), e o modelo usa o padrão BGR (azul, verde, vermelho). Podemos mudar esse padrão usando indexação de matrizes no Numpy: A sintaxe X[::-1] inverte a ordem de uma matrix (em determinada dimensão - ex: [1,2,3] passaria a ser [3,2,1])


In [ ]:
mean_img = params['mean image']

def process_dataset(x):
    x = np.transpose(x, (0,3,1,2))  # Modifica dados para: exemplos x canais RGB x altura x largura
    x = x[:, ::-1]                  # Modifica canais de RGB para BGR
    
    x = x - mean_img
    return x
    

In [ ]:
x_train_processed = process_dataset(x_train)
x_valid_processed = process_dataset(x_valid)

In [ ]:
x_train.shape

# Exercício - transfer learning

Possuímos agora a base de dados destino (tarefa de classificação das 5 classes) no formato correto para o modelo pré-treinado. 

Esse exercício consiste em fazer o transfer learning da base ImageNet para a base Caltech-5classes, usando o método DeCAF[[1](http://arxiv.org/abs/1310.1531)]:

![](images/oquab_decaf.svg.png)

Dado que a rede já está treinada, os passos serão:

1. Obter a saída da camada *fc7*, ao fazer forward-propagation da base de treinamento (x_train) e validação (x_valid)
2. Criar um modelo de regressão logística usando sklearn. Treinar usando os vetores de características ex
3. Avaliar a performance em validação, usando a representação na camada *fc7*


Dicas:

* Use ```lasagne.layers.get_output``` para obter a saída da rede em uma camada à sua escolha. Lembre de usar o parametro "deterministic=True"
* Obter as representações (forward-propagation) utiliza bastante memória, então é recomendável fazê-lo em mini-batches (isto é, obter a representação de alguns exemplos a cada vez). Para isso, após compilar uma função do theano para obter as representações, use a função ```get_output_batch``` definida abaixo para aplicá-la à toda a base de treino/validação: ```x_train_processed, x_valid_processed```

In [ ]:
#Roda uma função em batches e concatena o resultado:
def get_output_batch(function, x, batch_size=32):
    output = []
    for batch_start in xrange(0, len(x), batch_size):
        output.append(function(x[batch_start:batch_start+batch_size]))
    return np.vstack(output)


In [ ]:
# Sua solução

In [ ]:
%load solutions/transfer_decaf.py

Obtemos um resultado superior à 90% - uma boa melhora comparado ao modelo de regressão logística usando pixels!

# Visualizando as entradas classificadas incorretamente

## Exercício: Obtendo predições do modelo

Obtenha as predições do modelo treinado acima, para a base de dados de validação. 

Dica: utilize a função ```<classificador>.predict``` [link](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression.predict)


In [ ]:
# Sua solução

valid_preds = # codigo

In [ ]:
%load solutions/transfer_preds.py

In [ ]:
#Função para exibir uma imagem e a predição do modelo

def plot_img(X, y, ypred, ax=None):
    mean_img = params['mean image']
    if (ax == None):
        f, ax = plt.subplots()
        
    #Desfazendo o pré-processamento das imagens:
    img = X+mean_img
    img = img [::-1]
    img = np.transpose(img,[1,2,0])
    ax.imshow(img.astype(np.uint8))
    ax.set_title('Predicao do modelo: %s; correto: %s' % (classes[ypred], classes[y]))
    ax.axis('off')

In [ ]:
error_idx = np.flatnonzero(valid_preds != y_valid)

In [ ]:
f, ax = plt.subplots(len(error_idx), figsize=(12,8))
for i in range(len(error_idx)):
    idx = error_idx[i]
    plot_img(x_valid[idx], y_valid[idx], valid_preds[idx], ax[i])

## Exercício: Performance na base de teste

Avalie a performance na base de teste, seguindo os mesmos passos que usamos para validação:

1. Execute o pré-processamento nas imagens de teste
2. Use a rede pré-treinada para obter a representação na camada fc7
3. Use o modelo linear treinado para obter o score

In [ ]:
#Sua solução

In [ ]:
%load solutions/transfer_test.py

# Importante: antes de rodar a parte 2, feche esse notebook (File ->close and halt) para liberar a memória utilizada

# Exercícios extras:

* No exercício acima, usamos apenas uma divisão da base entre treinamento e validação. No caso de bases pequenas, uma estratégia mais robusta para estimar o erro chama-se k-fold cross-validation. Use Scikit-Learn para implementar esse tipo de validação ([manual](http://scikit-learn.org/stable/modules/cross_validation.html#cross-validation))

* Faça o treinamento utilizando a representação obtida em outras camadas (e.g. fc6)
